# Assigment 4

Welcome to assigment 4. Today you will not *implement* any new model (as promissed), but you will work with a new kind of data modality (text) and you will get a chance to try some new models: **Support Vector Machine** (SVM) and **Random Forest** (RF). 

These models can be found in the `sklearn` library which we used before. At the time of writing this library is one of the most used Python libraries related to Machine Learning, and considered the go-to place for an effective implementations of models like these.

In this assigment you will try to preprocess text data, *"fit"* them using these models and then you will have a chance to play with parameters of this model to obtain better performance in a process called *"fine-tuning"*.

As always, we'll start with some imports (we highly recomend you just run this cell and leave it as it is).

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from nltk.stem.porter import PorterStemmer
from matplotlib import pyplot as plt
from sklearn import svm
import numpy as np
import utils
import pandas as pd
%matplotlib notebook

### Part 1: SVM

First, we will look at **Support Vector Machines (SVM)**. In order to illustrate how they works we prepared a very simple toy dataset for a classification task. In this task we try to classify the data into one of two classes. As we mentioned many times before, it is good practice to visualize the data first. We can see that the *red* class has some outliers in the top left corner.

*Note: You can add your own points by clicking into the plot: left click for green points and right click for red points. If you want to remove outliers use* `show_plot('no_outliers')`.

In [ ]:
X, y, p = utils.show_plot()

Now lets try to fit a simple SVM with **linear** kernel to this data. This implementaion is based on library called `libsvm` which is probably the most often used implementation of a (linear) SVM and has bindings for virtually any programming language. The `fit()` function will return an `SVC` object. In a very handy way, `print`ing (in other words visualizing the representation of) this object gives us the list of all parameters of this classifier. 

In [ ]:
lin_svm = svm.SVC(C=1, kernel='linear')
lin_svm.fit(p.X, p.y)

_Note: we omitted the `print` statement in the previous cell for clarify._

We can again (like the last time) visualize the *decision boundary* of our classifier. We can see that with our current parameter setting we missclassified one of our outliers. This is due to the fact that the $C$ parameter is set to a fairly low value.

Informally, the C parameter is a real positive value that controls the penalty for misclassified training examples. A large $C$ parameter tells the SVM to try to classify all the examples correctly, despite the costs (pun intended -- we refer to the "real life" or testing costs here, since such a model would overfit the training data). $C$ plays a role similar to $\frac{1}{\lambda}$, where $\lambda$ is the regularization parameter similar to that in logistic regression.

In [ ]:
utils.show_boundary(p.X, p.y, lin_svm.predict)

Your task is now to find a value of $C$ that makes the classifier correctly classify all the points. You can also try lower values of C and see what effect does this have on the decision boundary, especially near the outliers. 

*Write here value of C that correcly classifies all the points *

C = 

---------------------------------------------------------------------------------------------------------------------

### Part 2: Gaussian kernel

We loaded a second toy example for you. As you can see in the plot below, the data is probably not going to be linearly separable. Thanks to the nature of the data, however, we can use a **Guassian kernel** (in `sklearn` this is often times represented as `'rbf'`, which stands for * **R**adial **B**asis **F**unction *) in combination with our linear SVM to perform a non-linear classification.

Intuitivelly, you can think of the Gaussian kernel as a similarity function/metric that measures the *"distance"* between a pair of examples. The Gaussian kernel is also parameterized by a bandwidth parameter $\gamma$, which determines how fast does the similarity metric decrease (down to 0), as the examples get further apart from the mean of the Gaussian that represents the kernel.

In [ ]:
data = np.loadtxt('sample_data2.csv', delimiter=',')
X = data[:, :2]
y = data[:, 2]

plt.figure()
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.coolwarm)
plt.show()

Your task is now to find a good fit of the data. You can try different values for $C$ and $\gamma$ parameters.

*Note: if no gamma is specified, `sklearn` uses the gamma='auto' setting, which sets $\gamma$ to  $\frac{1}{num\_features}$. This most probably won't work too well in your case and we expect you to try a few different values.*

Write here the values of $C$ and $\gamma$ for which you got the best fit:

C = 


$\gamma$ = 


In [ ]:
svc = svm.SVC(C=1, kernel='rbf')
svc.fit(X, y)

In [ ]:
utils.show_boundary(X, y, svc.predict)

---------------------------------------------------------------------------------------------------------------------


### Part 3: Lyrics-based song sentiment clasification

Let us now consider a more 'real world' example. Until now we worked with data that came from various measurements performed in the real world -- especially in the context of regression. We also worked with pictures. Another big area where Machine Learning has been used to a great success is text.

We prepared an interesting text dataset for you. It is made of song lyrics and your taks will be to classify the songs into two categories - *sad* or *happy*. When it comes to text in Machine Learning, there is quite a lot of preprocessing involved, as most of the time just directly feeding the words or characters into a simple classifier such as an SVM is not an option. So first, lets take a look at some of our data.

In [ ]:
df_train = pd.read_csv('train_lyrics_1000.csv')
df_test = pd.read_csv('valid_lyrics_200.csv')

df_train.tail()

In [ ]:
X_train = df_train['lyrics']
y_train = df_train['mood']

X_test = df_test['lyrics']
y_test = df_test['mood']

The data is already split into *train* and *test* sets. But right now our target variables `y_train` and `y_test` contain just the words 'sad' and 'happy'. This is not very desirable, as our model does not really *"speak the language of characters"* but would much more prefer to get the classes represented as numerical values. We need to fix this. 

To do so, please fill the body of function `get_label_vec`, which gets a vector of words (representing classes) and a dictionary of class-index mapping, and transforms this vector into a vector based on the provided dictionary. For example, if the vector `y = ['sad', 'happy', 'sad']` and `classes = {'sad': 1, 'happy': 0}` resulting vector should be `y = [1, 0, 1]`

In [ ]:
def get_label_vec(y, classes):
    pass

classes = {'sad': 1, 'happy': 0}
classes_inv = {1: 'sad', 0: 'happy'}
y_train = get_label_vec(y_train, classes)
y_test = get_label_vec(y_test, classes)

The next step in most text preprocessing pipelines is usually removal of frequent words, such as 'the', 'and', 'I' and so on. These words many times do not contain semantic value relevant to the task at hand. Since they are very frequent, they usually inflate text representations which may cause the model to focus on not that important or plainly worng features.

In the literature these words are also often times called *'stop words'*. In the next cell, please fill in the body of  the `remove_stop_words()` function which takes as parameters `data` (list of lyrics) and a list of `stop_words`, and should return a list of lyrics in which these stopwords would be removed.

*Note: as a default preprocessing step, it is most of the time a good idea to lower case all the words*

In [ ]:
def remove_stop_words(data, stop_words):
    pass

stop_words = np.loadtxt('stop_words.csv', delimiter=',', dtype='str')

X_train = remove_stop_words(X_train, stop_words)
X_test = remove_stop_words(X_test, stop_words)

Besides stop words, we can also find a lot of other words that contain little information related to the sentiment of a given song. For instace, 'hello', 'cup', 'jeans' and so on. We would like our representation to contain only words that have some semantic meaning relevant to our task. In order to make it so, we took a semantic lexicon (https://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html#lexicon) and kept only words that have 'positive' or 'negative' class assigned to them. 

Your task is now to fill the body of the `keep_semantic_words()` function which takes as parameters `data` (list of lyrics) and `semantic_words` (a list of relevant words). This function should again return a list of lyrics with only semanticly relevant words present in the lyrics text.

In [ ]:
def keep_semantic_words(data, semantic_words):
    pass

semantic_words = np.loadtxt('semantic_words.csv', delimiter=',', dtype='str')

X_train = keep_semantic_words(X_train, semantic_words)
X_test = keep_semantic_words(X_test, semantic_words)

The last preprocessing step of our pipeline is called **word stemming**. It is a process where words are reduced to their **stemmed form**. For example *discount*, *discounts*, *discounted* and *discounting* are all replaced with *discount*. Sometimes, the stemmer actually strips off additional characters from the end, so *include*, *includes*, *included*, and *including* are all replaced with *includ*. This should make the task a bit easier to the model, as it does not have to learn that words like *include* and *includes* have the same semantic meaning but are represented differently (we sometimes say that they have a different form).

In our case we are going to use Porter stemmer (https://tartarus.org/martin/PorterStemmer/) which can be found in the `nltk` library along with snowball stemmer (http://www.nltk.org/howto/stem.html).

In [ ]:
porter = PorterStemmer()
(porter.stem(x) for x in ['discounting', 'discounted', 'discounts', 'discount'])

In the next step/cell, please fill in the body of `stem_words()` function. This function takes `data` (list of lyrics) and `stem` which should be the stemming function of our Porter stemmer. It should return a list of lyrics with all words processed by a given stemming function.

In [ ]:
def stem_words(data, stem):
    pass

porter = PorterStemmer()
X_train = stem_words(X_train, porter.stem)
X_test = stem_words(X_test, porter.stem)

Now let's have a quick look at what has been left of our text after applying all this preprocessing.

In [ ]:
idx = np.random.randint(len(X_train), size=5)
for i in idx:
    print("{}: {}".format(classes_inv[y_train[i]], X_train[i]))

After we removed all unnecessary words without semantic meaning and helped the model with representation of words in the respective lyrics texts, we need to transorm our song lyrics into a format that the models we plan on using could actually consume -- in general this usually means putting it into some vector representation.

There are many representations we can choose from. One of the most popular right now are distributed representations (such as `word2vec`), but we are going to use a very simple representation called *bag-of-words*.

All of the preprocessing has left us with a specific volabulary. It contains 1388 words. We will construct vectors of size (`1388x1`) for each training example (in other words, each song). A vector like this will be filled with zeros in the begining. We will then go through the text of each training example and add one (keeping multiplicity) in this *"vocabulary vector"* to those places which represent words that can be found in the considered text.

For example, if we would have had vocabulary `{'first', 'second', 'third', 'fourth'}` and vectors `['first', 'second']`,  `['third', 'fourth']` and `['third', 'first', 'third']` we would have constructed vectors of size (4x1) - `[1, 1, 0, 0]`, `[0, 0, 1, 1]` and `[1, 0, 1, 0]`. Note that the order of words in the input vectors does not play a role in this representation.

In the next cell please fill in the body of the `transform_bag()` function, which should return a matrix of these vectors based on the `vocab` parameter. The `data` parameter is a list of lyrics.

In [ ]:
def transform_bag(data, vocab):
    pass

vocabulary = np.loadtxt('vocabulary.csv', delimiter=',', dtype='str').tolist()

X_train = transform_bag(X_train, vocabulary)
X_test = transform_bag(X_test, vocabulary)

Once we have some sort of a vector representation of the text, we can now try to train a sample SVM model.

Since we do not have a large dataset, you should implement K-fold cross-validation with `k=5` to help you find good hyper-parameters without overfitting on your training set. `sklearn` has a nice [utility function](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) that can help you with this a lot.

In [ ]:
svc = svm.SVC(C=1, kernel='rbf')
svc.fit(X_train, y_train)

print("Traning accuracy for SVM: {}".format(svc.score(X_train, y_train)))

In the next cell we can do the same with Random Forests. Please, also use this model in a cross-validation setting with `k=5`.

In [ ]:
rfc = RandomForestClassifier(n_estimators=3, max_depth=3)
rfc.fit(X_train, y_train)

print("Traning accuracy for Random Forest: {}".format(rfc.score(X_train, y_train)))

---------------------------------------------------

### Part 4 (optional): Random Forest visualization

In this section we will take a closer look at how the Random Forest classifier actually work by visualizing some of the trees that make a Random Forest we trained above.

Let us briefly review how does the training of Random Forests differ from ordinary tree bagging (bootstrap aggregation). Suppose we have a training set of features $X = x_1, \cdots, x_n $ along with the ground truth labels $Y = y_1, \cdots, y_n$. When bagging with decision trees, we would have done the following $B$ times (for $B$ trees):

1. Randomly sample $n$ samples (with replacement) from $X$ and $Y$. Call this sample $X_b$, $Y_b$
2. Train a decision tree on $X_b$ and $Y_b$.

This gives us $B$ trained decision trees. The final classification prediction can then be obtained by letting all of these decision trees classify the input and returning the majority opinion (also called "majority vote").

The difference between this "tree bagging" and Random Forests is a small change in the training process: the trees in a Random Forests consider a random subset of features at each candidate split. The reason for this is pretty simple: when you consider the ordinary "tree bagging" procedure mentioned above, it may easily happen that one or a few features will be strongly correlated with the output, and so they will be chosen in many of the $B$ trees. This will in turn cause them to become correlated. Introducing randomness into the mix helps the Random Forest generalize better. 

The process described above is sometimes also called "feature bagging". Note that this section is mostly a paraphrased version of the [Wikipedia entry](https://en.wikipedia.org/wiki/Random_forest) for Random forest (which is certainly worth reading).

In the next few cells we visualize some of the trees in our Random Forest. Note that the first line in the non-leave tree nodes (which is usually in the form of `word <= 0.5`) can be essentially interpreted as asking the question "Can `word` be found in the representation of lyrics of a given song?". The visualization also lets you very nicely see how is the prediction made in case of each tree and can also give you some intuition as to how are these trees built up.

In [ ]:
from sklearn import tree
import graphviz
estimator = rfc.estimators_[0]
dot_data = tree.export_graphviz(estimator, out_file=None,
                                    feature_names=vocabulary, 
                                    class_names=['happy', 'sad'], 
                                    rounded=True, filled=True)
graph = graphviz.Source(dot_data)
graph

In [ ]:
estimator = rfc.estimators_[1]
dot_data = tree.export_graphviz(estimator, out_file=None,
                                    feature_names=vocabulary, 
                                    class_names=['happy', 'sad'], 
                                    rounded=True, filled=True)
graph = graphviz.Source(dot_data)
graph

In [ ]:
estimator = rfc.estimators_[2]
dot_data = tree.export_graphviz(estimator, out_file=None,
                                    feature_names=vocabulary, 
                                    class_names=['happy', 'sad'], 
                                    rounded=True, filled=True)
graph = graphviz.Source(dot_data)
graph

### Part 5 (optional): For bonus points

In this section we'll briefly describe a few things which may be worth trying out and awarded a few bonus points.

Please submit your "solutions" to these "challenges" in this notebook (which needs to be fully filled out). Feel free to create new cells below.

In the following sections you can find the actual "challenges" along with the amount of points you can get for solving them.

#### Baseline (max 1 point)

When we compare Machine Learning models, it is usually helpful to have a baseline to compare any of your models against. A baseline is a very simple model -- often times the simplest one you can think of. 

For instance for regression, the baseline may be a model that predicts the average of all ground truth values seen in the training data, regardless of the inputs. For classification a very simple baseline would be a model that predicts the majority (most often occuring) class, no matter what inputs it gets.

Once we have the baseline, we can use it to interpret any other results. Suppose for instance that we would have obtained baseline classification accuracy of 57% and our new model would achieve 84%. We can interpret this result as our new model being able to achieve as much as 62% error reduction.

In this challenge we ask you to do two things:

1. Prepare a simple baseline for the lyricis sentiment classification task we deal with in this assignment
2. Compare your best model to the baseline (in terms of prediction accuracy -- feel free to use the error reduction approach described above)

#### Does removal of "non-semantic" words/stemming actually help? (max 1 point)

In the text above we removed non-semantic words and stemmed them, which may or may not actually make sense in a real world scenario.

For bonus points show what sort of an effect would skipping stemming and/or not remoivng "non-semantic" words have on the final performance of the model.


#### Improve classification using other variables from the dataset! (max 2 points)

Note that in the example above we did not make use of the `title`, `artist`, `genre` or `year` fields that can be found in the dataset we worked with.

For bonus points try to make use of this data in a creative way -- it may be that you'll find out that some `year`/`artist`/`genre`/`year` has a tendecny to make a song happy/sad.


#### Do something cool! (max 3 points)

Try different kernels/parameters for the SVM/Random Forest, document their effect and find the best fit of the data you can get. 

Try adding some new features, for example instead of bag-of-words try to use tf-idf (https://en.wikipedia.org/wiki/Tf%E2%80%93idf) -- (feel free to use the implementation in [scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html).  Can you "extract" some "classification rules" that intuitively make sense from some of the Random Forests you trained? 

Do something else (with the provided data and using Machine Learning models discussed in this assignment) that you think the TAs would consider to be cool. Pleasant surprises in the form of your answers may cause pleasant surprises in form of the bonus points you get.